In [1]:

import xarray as xr
import numpy as np
import datetime as dt
import datetime_tools as dtools
import month_indices as mi
import findlatlonbox as findbox
import data_extend 
from select_times import select_times
from data_from_dates import data_from_dates
from box_data import box_data
from single_list import single_list
from box_coords import box_coords

In [2]:
# user generated dependencies (included in folder) are:
    # new datetime_tools.py
    # new month_indices.py functions
    # findlatlon.py
    # findlatlonbox.py
    # select_times.py
    # data_extend.py
    # data_from_dates.py
    # box_data.py
    # single_list.py

# uses block_centroids_NH_array.nc and yearly block data 
# with title format: "all_blocks_2by2pt5_v2_era5_2001.nc"

In [3]:
# CHOICES

# choose how far to extend box on each side (def = 15)
sz = 15

# right now, using generic lat/lon as input for box coordinate retrieval
# make generic lat/lon
latg = np.arange(-89.5,90.5)
long = np.arange(0.5,360.5)

# filepaths
# centroid path
cpath = '/Volumes/T7/Data/block-centroids/'
# NH file name
fname = 'Block_centroids_CORRECTED_NH_array.nc'

# datafile paths - MUST include year as {year} for fxn to work
# block path
# fpath = '/Volumes/T7/Data/all_blocks_2by2pt5_v2_era5_{year}.nc'
# cloud data plath (anomalies)
fpath = '/Volumes/T7/yearly_cloud_data_anom/{year}/anom_CERES_{year}_cldfrac.nc'

# choose path to save figures 
svpath = 'figures/'



In [4]:
# DATES
# open centroid file
ds = xr.open_dataset(cpath + fname)
dates = ds.dateindex.values

# use fxn I made to put dates in one list and get rid of nans
dates = single_list(dates)

# get dates assuming date units are "Days since Jan 1 1980"
start_date = dt.datetime(1980,1,1)
# using function I created
drange = dtools.give_enddate(start_date, dates)

# get index of first centroid in 2001 (only cloud data starting in 2001) 
st = select_times(drange, 2001)[0]
print(st)

# keep only dates from 2001
drange = drange[st:]



5496


In [5]:
# use cloud data (fpath) and drange to get an array of the cloud data on dates
# in drange!
sel_data = data_from_dates(drange,fpath)
print(sel_data.shape)

(5250, 180, 360)


In [6]:
# get centroid lat/lons and combine into one array
lonv = ds.clon.values
latv = ds.clat.values
# put all lats & lons in one long list without nans:
# and keep only ones 2001 and after
lats,lons = [single_list(x)[st:] for x in [latv,lonv]]
# stack them so you've got a 2d array of dims: len(lats),2
cents = np.stack((lats,lons),1)
print(cents.shape)



(5250, 2)


In [7]:
# BOX CORNER RETRIEVAL - using generic latg/long created at start
# using function I wrote, output is list of arrays
# [south index, north index, west index, east index]
corners,latext,lonext = findbox.corners(latg,long,cents,sz)
print(corners.shape)

(5250, 4)


In [8]:
# get the data using sz, corners and the selected days of data
# the fxn extends the days of data and takes each box
box_data = box_data(corners,sel_data, sz)
print(box_data.shape)

(5250, 31, 31)


In [12]:
# use function i wrote to input extended lat and lon grids and the corner 
# indices and get the lat/lon coords of the box corners
# box_coords are south lat, norther lat, west lon, easter lon
# latext & lonext output from box corner fxn, no need to extend arrays again

box_coords = box_coords(latext, lonext, corners)
print(box_coords.shape)
[print(box_coords[x], cents[x]) for x in range(5)]

(5250, 4)
[ 45.5  75.5 341.5 371.5] [ 61.   356.25]
[ 45.5  75.5 -13.5  16.5] [61.    1.25]
[43.5 73.5 -8.5 21.5] [59.    6.25]
[43.5 73.5 -8.5 21.5] [59.    6.25]
[43.5 73.5 -8.5 21.5] [59.    6.25]


[None, None, None, None, None]